# Building Classes

## Terminology

###### class
Tell Python to make a new type of thing.
###### object
Two meanings: the most basic type of thing, and
any instance of some thing.
###### instance
What you get when you tell Python to create a class.
###### def
How you define a method of a class.
###### self
Inside the methods in a class, self is a variable for
the instance/object being accessed.
###### inheritance
The concept that one class can inherit traits from
another class, much like you and your parents.
###### attribute
A property that classes have that are from composition
and are usually variables.
###### is-a
A phrase to say that something inherits from another, as in
a "salmon" is-a "fish."

## What am I really missing out on by not writing classes?

### 1. More convenient collections of fields

You might end up with lots of `list`s or `dict`s that share the same keys to access different kinds of data associated with a single logical (geographical) object:
```python
obs_temperature[42] = 20  # Temperature at the observational point No. 23
obs_humidity[42] = 75  # Humidity at the observational point No. 23
obs_name[42] = 'research_vessel'  # Name of the obs. point No. 23
```

By switching to classes you could have a single list of objects, each of which has several named fields on it to address the associated data:
```python
# Everyting at the point No. 42
obs[42].temperature = 20
obs[42].humidity = 75
obs[42].name = 'research_vessel'
```
Now you can keep all of the fields under one roof, which makes accessing and passing these objects around much more convenient.

It's also easier to pass around big tuples of stuff from function to function.

*Note: we will have a look at Python's `namedtuples` in a future session.*

### 2. The ability to chain objects together and let them interact in an expressive way

### 3. Domain-specific models

### 4. Custom Exceptions

## Special methods (aka Magic methods)

* A class can implement certain operations that are invoked by special syntax (such as arithmetic operations or subscripting and slicing) by defining methods with special names. This is Python's approach to operator overloading, allowing classes to define their own behavior with respect to language operators.
* For instance, if a class defines a method named `__getitem__()`, and `x` is an instance of this class, then `x[i]` is roughly equivalent to `type(x).__getitem__(x, i)`.
* Except where mentioned, attempts to execute an operation raise an exception when no appropriate method is defined (typically `AttributeError` or `TypeError`).

### Note dunder-methods vs private attributes

In [ ]:
class Car:
    def __init__(self, name, color):
        self.name = name
        self.__color = color
        self.__blah__ = name.upper()
        
#     @property
#     def color(self):
#         return self.__color

In [ ]:
a = Car('aaa', 'bbb')

examples: Lagranto, aospy, HPC python book...

### `__str__` vs `__repr__`

* You can control to-string conversion in your own classes using the `__str__` and `__repr__` "dunder" methods.
Writing your own Java-esque "tostring" methods is considered unpythonic.
* The result of the `__str__` method should be readable. The result of  `__repr__` should be unambiguous.
* You should always add a `__repr__` to your classes. The default implementation for `__str__` just calls `__repr__` internally, so by implementing `repr` support you'll get the biggest benefit.

#### Useful trick to avoid repetition
* use the object's `__class__.__name__` attribute, which will always reflect the class' name as a string.

```python
def __repr__(self):
    return (f'{self.__class__.__name__}('
            f'{self.color!r}, {self.mileage!r})')
```

## References
* https://dbader.org/blog/6-things-youre-missing-out-on-by-never-using-classes-in-your-python-code
* https://dbader.org/blog/python-repr-vs-str
* https://dbader.org/blog/python-dunder-methods
* https://docs.python.org/3.6/reference/datamodel.html
* http://www.ceda.ac.uk/static/media/uploads/ncas-reading-2015/20_ceda-oop.pdf